In [28]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
cd ../TileEM/

/home/jlee782/crowd-seg/analysis/TileEM


In [10]:
cd ../pixel_em/

/home/jlee782/crowd-seg/analysis/TileEM/pixel_em


In [12]:
import pandas as pd

In [47]:
PR_tbl = pd.read_csv("full_PR_table.csv")

In [15]:
PR_tbl.mean()

num_workers     12.333333
sample_num       2.933333
objid           24.000000
MV_precision     0.923356
MV_recall        0.927232
EM_precision     0.922656
EM_recall        0.928828
dtype: float64

In [66]:
PR_pixelEM = pd.read_csv("full_PR_table.csv")

In [67]:
thres_lst = list(set(PR_tbl.thresh))

In [73]:
for thresh in sorted(thres_lst):
    print "Thres:",thresh

    PR_pixelEMi =PR_pixelEM[(PR_pixelEM["thresh"]==thresh)]
    PixelEMP= PR_pixelEMi["EM_precision"].mean()

    PixelEMR= PR_pixelEMi["EM_recall"].mean()
    #print len(PR_pixelEMi["EM_recall"])
    print PixelEMP,PixelEMR

Thres: -4
0.907867744268 0.942336226818
Thres: -2
0.918493037599 0.933428418656
Thres: 0
0.922655837765 0.928828076539
Thres: 2
0.926256715301 0.924138549868
Thres: 4
0.934107514295 0.912167566261


In [71]:
cd ../

/home/jlee782/crowd-seg/analysis/TileEM


In [72]:
PR_pixelEM = pd.read_csv("pixel_em/full_PR_table.csv")
PR_pixelEMi =PR_pixelEM[(PR_pixelEM["num_workers"]==Nworker)&(PR_pixelEM["sample_num"]==batch_num)&\
(PR_pixelEM["objid"]==objid)&(PR_pixelEM["thresh"]==thresh)]
PixelEMP= PR_pixelEMi["EM_precision"] 
PixelEMR= PR_pixelEMi["EM_recall"]
print PixelEMP,PixelEMR

NameError: name 'Nworker' is not defined

In [29]:
PR_tbl.mean()

num_workers     11.265331
sample_num       3.104163
objid           23.842114
thresh           0.108136
MV_precision     0.924038
MV_recall        0.925455
EM_precision     0.921843
EM_recall        0.926570
dtype: float64

- compute jaccard